# Multi-wavelength maps

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import starry

starry.config.lazy = False
starry.config.quiet = True

Instantiate a map with 100 wavelength bins:

In [ ]:
map = starry.Map(ydeg=10, nw=100)

Just for fun, give it an amplitude that varies sinusoidally with wavelength:

In [ ]:
wav = np.linspace(0, 1, map.nw)
map.amp = np.sin(2 * np.pi * wav)

Visualize it:

In [ ]:
map.show()
plt.plot(wav, map.flux().reshape(-1))

In [ ]:
from ipywidgets import widgets

out1 = widgets.Output(layout={})
out2 = widgets.Output(layout={})

with out1:
    map.show()
    plt.close()

with out2:
    fig = plt.figure(figsize=(6, 3))
    plt.plot(wav, map.flux().reshape(-1))
    plt.xlabel("wavelength")
    plt.ylabel("total flux")
    display(fig)
    plt.close()

widgets.HBox([out1, out2])